In [1]:
import matplotlib.pyplot as plt
import wntr
import wntr.network.controls as controls
import sqlite3
import csv
import time
import pandas as pd

In [2]:
# Create the network
inp_file = '../EPANET_topologies/wadi_run_55.inp'
wn = wntr.network.WaterNetworkModel(inp_file)

# Set option for step-by-step simulation
wn.options.time.duration = 840
wn.options.time.hydraulic_timestep = 1
wn.options.time.pattern_timestep = 1
wn.options.time.report_timestep = 1
wn.options.time.quality_timestep = 5 

list_header = []
node_list = list(wn.node_name_list)
link_list = list(wn.link_name_list)

In [9]:
reservoir_list = get_node_list_by_type(node_list, 'Reservoir')
tank_list      = get_node_list_by_type(node_list, 'Tank')
junction_list  = get_node_list_by_type(node_list, 'Junction')
pump_list      = get_link_list_by_type(link_list, 'Pump')
valve_list     = get_link_list_by_type(link_list, 'Valve')

list_header = ["Timestamps"]
aux = create_node_header(tank_list)
list_header.extend(aux)

aux = create_node_header(reservoir_list)
list_header.extend(aux)

aux = create_link_header(pump_list)
list_header.extend(aux)

aux = create_link_header(valve_list)
list_header.extend(aux)

dd_results_list = []
dd_results_list.append(list_header)

print(list_header)

['Timestamps', 'T1_LEVEL', 'T2_LEVEL', 'T0_LEVEL', 'SWaT_LEVEL', 'PUB_LEVEL', 'P_RAW1_FLOW', 'P_RAW1_STATUS', 'P_RAW2_FLOW', 'P_RAW2_STATUS', 'P_B2_FLOW', 'P_B2_STATUS', 'P_B1_FLOW', 'P_B1_STATUS', 'V_Gi_G_FLOW', 'V_Gi_G_STATUS', 'V_Gi_B_FLOW', 'V_Gi_B_STATUS', 'V_SWaT_FLOW', 'V_SWaT_STATUS', 'V_PUB_FLOW', 'V_PUB_STATUS', 'V_ER2i_FLOW', 'V_ER2i_STATUS', 'V_ER2o_FLOW', 'V_ER2o_STATUS', 'V_ER1i_FLOW', 'V_ER1i_STATUS', 'V_ER1o_FLOW', 'V_ER1o_STATUS', 'FCV_ER_FLOW', 'FCV_ER_STATUS', 'FCV_RWin_FLOW', 'FCV_RWin_STATUS']


In [4]:
def get_node_list_by_type(list, type):
    result = []
    for node in list:
        if wn.get_node(node).node_type == type:
            result.append(str(node))
    return result

In [5]:
def get_link_list_by_type(list, type):
    result = []
    for link in list:
        if wn.get_link(link).link_type == type:
            result.append(str(link))
    return result

In [6]:
def create_node_header(list):
    result = []
    for node in list:
        result.append(node+"_LEVEL")
    return result

In [7]:
def create_link_header(list):
    result = []
    for link in list:
        result.append(link+"_FLOW")
        result.append(link+"_STATUS")
    return result

In [3]:
#v_pub_condition_initial = controls.SimTimeCondition(wn, '=', '00:00:00')
v_pub_condition_open = controls.ValueCondition(wn.get_node("T0"), 'level', '<', 0.3008)
v_pub_condition_closed = controls.ValueCondition(wn.get_node("T0"), 'level', '>', 0.576)

#v_raw1_condition_initial = controls.SimTimeCondition(wn, '=', '00:00:00')
v_raw1_condition_closed_1 = controls.ValueCondition(wn.get_node("T0"), 'level', '<', 0.3008)
v_raw1_condition_open = controls.ValueCondition(wn.get_node("T2"), 'level', '<', 0.08)
v_raw1_condition_closed_2 = controls.ValueCondition(wn.get_node("T2"), 'level', '>', 0.36)

#v_er2i_condition_initial = controls.SimTimeCondition(wn, '=', '00:00:00')
v_er2i_condition_closed  = controls.ValueCondition(wn.get_node("T2"), 'level', '>', 0.36)
v_er2i_condition_open    = controls.ValueCondition(wn.get_node("T2"), 'level', '<', 0.08)

v_pub_open   = controls.ControlAction(wn.get_link('V_PUB'), 'status', 1)
v_pub_closed = controls.ControlAction(wn.get_link('V_PUB'), 'status', 0)

v_raw1_open   = controls.ControlAction(wn.get_link('P_RAW1'), 'status', 1)
v_raw1_closed = controls.ControlAction(wn.get_link('P_RAW1'), 'status', 0)

v_er2i_open   = controls.ControlAction(wn.get_link('V_ER2i'), 'status', 1)
v_er2i_closed = controls.ControlAction(wn.get_link('V_ER2i'), 'status', 0)

v_pub_control_open = controls.Control(v_pub_condition_open, v_pub_open, name='v_pub_control_open')
v_pub_control_closed = controls.Control(v_pub_condition_closed, v_pub_closed, name='v_pub_control_closed')

v_raw1_control_open = controls.Control(v_raw1_condition_open, v_raw1_open, name='v_raw1_control_open')
v_raw1_control_closed_1 = controls.Control(v_raw1_condition_closed_1, v_raw1_closed, name='v_raw1_control_closed_1')
v_raw1_control_closed_2 = controls.Control(v_raw1_condition_closed_2, v_raw1_closed, name='v_raw1_control_closed_2')

v_er2i_control_open = controls.Control(v_er2i_condition_open, v_er2i_open, name='v_er2i_control_open')
v_er2i_control_closed = controls.Control(v_er2i_condition_closed, v_er2i_closed, name='v_er2i_control_closed')

In [39]:
wn.get_link('V_PUB').status = 0
wn.get_link('P_RAW1').status = 0
wn.get_link('V_ER2i').status = 0

sim = wntr.sim.WNTRSimulator(wn) 

# START STEP BY STEP SIMULATION
iteration = 0
while iteration <= wn.options.time.duration:
    
    results_dd = sim.run_sim(convergence_error=True)

    # We should generate this automatically
    #['Timestamps', 'T1_LEVEL', 'T2_LEVEL', 'T0_LEVEL', 'SWaT_LEVEL', 'PUB_LEVEL', 'P_RAW1_FLOW', 'P_RAW1_STATUS', 'P_RAW2_FLOW',
    #'P_RAW2_STATUS', 'P_B2_FLOW', 'P_B2_STATUS', 'P_B1_FLOW', 'P_B1_STATUS', 'V_Gi_G_FLOW', 'V_Gi_G_STATUS', 'V_Gi_B_FLOW', 
    #'V_Gi_B_STATUS', 'V_SWaT_FLOW', 'V_SWaT_STATUS', 'V_PUB_FLOW', 'V_PUB_STATUS', 'V_ER2i_FLOW', 'V_ER2i_STATUS', 
    #'V_ER2o_FLOW', 'V_ER2o_STATUS', 'V_ER1i_FLOW', 'V_ER1i_STATUS', 'V_ER1o_FLOW', 'V_ER1o_STATUS', 'FCV_ER_FLOW', 
    #'FCV_ER_STATUS', 'FCV_RWin_FLOW', 'FCV_RWin_STATUS']
    
    values_list = []
    
    values_list.extend([results_dd.timestamp, wn.get_node("T0").level, wn.get_node("T1").level, wn.get_node("T2").level, 
                        wn.get_node("R_SWaT").level, wn.get_node("R_PUB").level])
    
    for junction in junction_list:
        values_list.extend([wn.get_node(junction).head - wn.get_node(junction).elevation])

    for pipe in pipe_list:        
    values_list.extend([pump1.flow, pump2.flow, pump1.status, pump2.status])

    dd_results_list.append(values_list)    
    iteration += 1

IndentationError: expected an indented block (<ipython-input-39-fe7c91f922b0>, line 31)